# Describing columns and treating null values

# Index

## 1. Data loading and first approach

* **1.1** Load data
* **1.2** Preliminary null value exploration
* **1.3** Preliminary column exploration
    

## 2. Null value treatment

* **2.1** Treating "Training Score" nulls (`avg_training_score`)
* **2.2** Treating "Rating" nulls (`previous_year_rating`)
* **2.3** Treating "Education" nulls (`education`)



In [19]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from sklearn.impute import SimpleImputer


# 1. Data loading and first approach

### 1.1 Load data

In [5]:
df = pd.read_csv('../data/trabajo1.csv')
df.head(3)

,employee_id,department,region,education,gender,recruitment_channel,no_of_trainings,age,previous_year_rating,length_of_service,awards_won,avg_training_score,is_promoted
0,65438,Sales & Marketing,region_7,Master's & above,f,sourcing,1,35,5.0,8,0,49.0,0
1,65141,Operations,region_22,Bachelor's,m,other,1,30,5.0,4,0,60.0,0
2,7513,Sales & Marketing,region_19,Bachelor's,m,sourcing,1,34,3.0,7,0,50.0,0


In [6]:
df.shape

(54808, 13)

In [7]:
df.describe().round(1)

,employee_id,no_of_trainings,age,previous_year_rating,length_of_service,awards_won,avg_training_score,is_promoted
count,54808.0,54808.0,54808.0,50684.0,54808.0,54808.0,52248.0,54808.0
mean,39195.8,1.3,34.8,3.3,5.9,0.0,63.7,0.1
std,22586.6,0.6,7.7,1.3,4.3,0.2,13.5,0.3
min,1.0,1.0,20.0,1.0,1.0,0.0,39.0,0.0
25%,19669.8,1.0,29.0,3.0,3.0,0.0,51.0,0.0
50%,39225.5,1.0,33.0,3.0,5.0,0.0,60.0,0.0
75%,58730.5,1.0,39.0,4.0,7.0,0.0,77.0,0.0
max,78298.0,10.0,60.0,5.0,37.0,1.0,99.0,1.0


 ### 1.2 Preliminary null value exploration
 

In [8]:
df.isnull().values.any()

True

What is the percentage of null values?

In [9]:
n_null =  df.isnull().values.sum()
n_null

9093

In [10]:
print(str(round(n_null/df.count().sum() * 100,4)) + "% of entries are null values")

1.2927% of entries are null values


In [11]:
df.isnull().sum()

employee_id                0
department                 0
region                     0
education               2409
gender                     0
recruitment_channel        0
no_of_trainings            0
age                        0
previous_year_rating    4124
length_of_service          0
awards_won                 0
avg_training_score      2560
is_promoted                0
dtype: int64

NULLS are located in columns where it makes sense to see them. We will have to decide how to treat each one of them. First impressions suggest:
* education: nulls represent no education
* previous_year_rating: nulls represent no prior ratings
* avg_training_socre: nulls represent no training score

### 1.3 Preliminary column exploration

In [12]:
df.nunique()

employee_id             54808
department                  9
region                     34
education                   3
gender                      2
recruitment_channel         3
no_of_trainings            10
age                        41
previous_year_rating        5
length_of_service          35
awards_won                  2
avg_training_score         59
is_promoted                 2
dtype: int64

**Further conclusions regarding a few of the columns**

All employee numbers are unique, there are no repeated employees:

In [13]:
df['employee_id'].nunique() == df.shape[0]

True

Some stats regarding some relevant columns

In [14]:
df[['age','length_of_service', ]].describe()[1:].round(2) # not interested in count

,age,length_of_service
mean,34.80,5.87
std,7.66,4.27
min,20.00,1.00
25%,29.00,3.00
50%,33.00,5.00
75%,39.00,7.00
max,60.00,37.00


In [12]:
df[['previous_year_rating','no_of_trainings', 'avg_training_score', 'awards_won' ]].describe()[1:].round(2)

,previous_year_rating,no_of_trainings,avg_training_score,awards_won
mean,3.33,1.25,63.71,0.02
std,1.26,0.61,13.52,0.15
min,1.00,1.00,39.00,0.00
25%,3.00,1.00,51.00,0.00
50%,3.00,1.00,60.00,0.00
75%,4.00,1.00,77.00,0.00
max,5.00,10.00,99.00,1.00


In [13]:
df['no_of_trainings'].min()

1

**IMPORTANT:** There are no entries where `no_of_trainings` = 0. This indicates that the `Na`values from `average_training_score` **do not** correspond with people that have not done any trainings. This will be treated further on.

## 2. Null value treatment

* **2.1** Treating "Training Score" nulls (`avg_training_score`)
* **2.2** Treating "Rating" nulls (`previous_year_rating`)
* **2.3** Treating "Education" nulls (`education`)


### 2.1 Treating "Training Score" nulls (`avg_training_score`)

In [21]:
df.isnull().sum()

employee_id                0
department                 0
region                     0
education               2409
gender                     0
recruitment_channel        0
no_of_trainings            0
age                        0
previous_year_rating    4124
length_of_service          0
awards_won                 0
avg_training_score      2560
is_promoted                0
dtype: int64

In [22]:
fig = px.histogram(df, x="avg_training_score")
fig.show()

Hay 59 valores nulos, se imputarán a la mediana

In [29]:
imp_mean = SimpleImputer(missing_values = np.nan, strategy = 'median')
imp_mean.fit(df['avg_training_score'].values.reshape(-1, 1)) # El .values.reshape(-1,1) es apra que encajen las dimensiones
df['avg_training_score'] =imp_mean.transform(df['avg_training_score'].values.reshape(-1,1))

In [28]:
df.isnull().sum()

employee_id                0
department                 0
region                     0
education               2409
gender                     0
recruitment_channel        0
no_of_trainings            0
age                        0
previous_year_rating    4124
length_of_service          0
awards_won                 0
avg_training_score         0
is_promoted                0
dtype: int64

### 2.2 Treating "Rating" nulls (`previous_year_rating`)

In [30]:
fig = px.histogram(df, x="previous_year_rating")
fig.show()

Por el mismo razonamiento que en el caso anterior, se imputarán a la mediana que es 3

In [31]:
imp_mean = SimpleImputer(missing_values = np.nan, strategy = 'median')
imp_mean.fit(df['previous_year_rating'].values.reshape(-1, 1)) # El .values.reshape(-1,1) es apra que encajen las dimensiones
df['previous_year_rating'] =imp_mean.transform(df['previous_year_rating'].values.reshape(-1,1))

In [32]:
df.isnull().sum()

employee_id                0
department                 0
region                     0
education               2409
gender                     0
recruitment_channel        0
no_of_trainings            0
age                        0
previous_year_rating       0
length_of_service          0
awards_won                 0
avg_training_score         0
is_promoted                0
dtype: int64

### 2.3 Treating "Education" nulls (`education`)

In [43]:
df["education"].value_counts()

Bachelor's          36669
Master's & above    14925
Below Secondary       805
Name: education, dtype: int64

In [39]:
df["education"].value_counts().values

array([36669, 14925,   805], dtype=int64)

In [42]:
list(df["education"].unique())

["Master's & above", "Bachelor's", nan, 'Below Secondary']

In [44]:
fig = px.bar(df, x=["Bachelor's", "Master's & above", "Below Secondary"], y=df["education"].value_counts().values)
fig.show()

En educación los NAN son importantes, se imputarán a un valor de string NA.

In [45]:
df["education"].fillna(value = "NA", inplace = True)

In [46]:
df.to_pickle('../data/df_treated_nulls.pkl')